In [41]:
# importing the keras librareis and packages

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Softmax
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

In [ ]:
import splitfolders
splitfolders.ratio("featured", output="dataset", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

Copying files: 792 files [00:02, 353.54 files/s]


In [46]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)

val_datagen = ImageDataGenerator(
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/identity/dataset/train',
    target_size=(299, 299),
    batch_size=30,
    class_mode='binary')

val_set = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/identity/dataset/val',
    target_size=(299, 299),
    batch_size=5,
    class_mode='binary')

Found 630 images belonging to 10 classes.
Found 75 images belonging to 10 classes.


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
base_model = Xception(weights="imagenet", include_top=False,
                      input_shape=(299, 299, 3))

base_model.layers.pop()
  
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_54[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

####Different batch sizes
|Batch size|Training accuracy|
|-|-|
|30|0.1159|
|60|0.1286|

#### Xception preprocessing
|Preprocessing|Training accuracy|
|-|-|
|No|0.1159|
|Yes|0.1381|

In [60]:
import matplotlib.pyplot as plt
for layer in base_model.layers:
    layer.trainable=False

#optimizer = SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
optimizer = Adam(learning_rate=0.0005)
model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer, metrics=["accuracy"])
epochs = 5

for epoch in range(epochs):
  print(epoch)
  training = model.fit(training_set, validation_data=val_set, epochs=10)
  
  if (epoch % 5 == 0):
    model.save('model')

model.save('model')

0
Epoch 1/10
 2/21 [=>............................] - ETA: 3:14 - loss: 18.9364 - accuracy: 0.1667

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy

# summarize history for accuracy
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()